# ***Clustering sobre noticias del periodico: El Tiempo (COL)***

Deseamos saber los temas de las noticias, asi como un resumen de estos en busca de una actualizacion via este periodo Colombiano. Para esto vamos a seguir estos pasos:

1. Obtencion de urls de las noticias.
2. Obtencion del texto de la noticia.
3. Generacion de embeddings de las noticias con [`Ollama`](https://ollama.com/) y la libreria [`langchain`](https://python.langchain.com/docs/introduction/).
4. Metodos de clustering.
5. Uso de LLM local (en Colab con Ollama) para obtener resumenes, insight y caractetizaciones de los cluster formados.
6. Discusion de resultados.


In [1]:
import requests
import pandas as pd
from io import StringIO

## ***Obtener urls de noticias del Tiempo***

In [2]:
url_new = (
    """https://www.eltiempo.com/deportes/otros-deportes/david-alonso-imparable-en-moto3-gano-en-indonesia-y-podria-asegurar"""
    """-el-titulo-en-la-proxima-carrera-en-japon-3385519"""
)

session = requests.Session()
# Peticion HTTP
response = session.get(url_new)
# Respuesta peticion
print("Respuesta a la peticion url:", response.status_code)
print(f"{100 * '='}")
# Contenido de la pagina web
print(response.text)

Respuesta a la peticion url: 200
<!DOCTYPE html>
    <html lang="es">
    <head>
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
                                    

                                            
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta name="viewport" content="width=device-width, initial-scale=1">
<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
<title>David Alonso sigue imparable en Moto3: ganó en Indonesia y podría asegurar el título en la próxima carrera, en Japón</title>
<link href="https://www.eltiempo.com/deportes/otros-deportes/david-alonso-imparable-en-moto3-gano-en-indonesia-y-podria-asegurar-el-titulo-en-la-proxima-carrera-en-japon-3385519" rel="canonical">
<link rel="alternate" href="https://www.eltiempo.com/deportes/otros-deportes/david-alonso-imparable-en-moto3-gano-en-indonesia-y-podria-asegurar-el-titulo-en-la-proxima-carrera-en-japon-3385519" hreflang="es-co"/>
<meta name

In [3]:
url = 'https://www.eltiempo.com/sitemap-articles-current.xml'
session = requests.Session()
response = session.get(url)
if response.raise_for_status:
    print(f"OK, with the page: {url}. Status Code: {response.status_code}\n")
    print(f"Text from Page:")
    print(response.text[: 1000])
else:
    print(f"Problem in the page: {url}")

OK, with the page: https://www.eltiempo.com/sitemap-articles-current.xml. Status Code: 200

Text from Page:
<?xml version="1.0" encoding="UTF-8"?>
<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9" xmlns:video="http://www.google.com/schemas/sitemap-video/1.1" xmlns:news="http://www.google.com/schemas/sitemap-news/0.9">
 <url>
  <loc>https://www.eltiempo.com/mundo/eeuu-y-canada/compre-en-walmart-y-me-obligaron-a-pasar-por-estos-3-filtros-3385099</loc>
  <lastmod>2024-09-29T16:00:00-05:00</lastmod>
  <news:news>
   <news:publication>
    <news:name>ElTiempo.com</news:name>
    <news:language>es</news:language>
   </news:publication>
   <news:title>'Compré en Walmart de EE. UU. y me obligaron a pasar por estos 3 filtros'</news:title>
   <news:publication_date>2024-09-29T16:00:00-05:00</news:publication_date>
   <news:keywords>compré, en, walmart, y, me</news:keywords>
  </news:news>
 </url>
 <url>
  <loc>https://www.eltiempo.com/justicia/investigacion/alias-zeus-la-caida-del-mayo

In [4]:
namespaces = {
    "ns": "http://www.sitemaps.org/schemas/sitemap/0.9",
    "news": "http://www.google.com/schemas/sitemap-news/0.9",
    "video": "http://www.google.com/schemas/sitemap-video/1.1"
}

# Hacemos lectura del texto XML
df_urls_news_el_tiempo = pd.read_xml(StringIO(response.text), xpath=".//ns:url", namespaces=namespaces)
df_urls_news_el_tiempo

,loc,lastmod,news
0,https://www.eltiempo.com/mundo/eeuu-y-canada/c...,2024-09-29T16:00:00-05:00,\n
1,https://www.eltiempo.com/justicia/investigacio...,2024-09-29T15:58:31-05:00,\n
2,https://www.eltiempo.com/bogota/secretaria-de-...,2024-09-29T15:57:18-05:00,\n
3,https://www.eltiempo.com/colombia/santander/du...,2024-09-29T15:54:50-05:00,\n
4,https://www.eltiempo.com/deportes/futbol-inter...,2024-09-29T15:53:07-05:00,\n
...,...,...,...
8281,https://www.eltiempo.com/vida/tendencias/copen...,2024-09-01T00:00:00-05:00,\n
8282,https://www.eltiempo.com/politica/gobierno/cri...,2024-09-01T00:00:00-05:00,\n
8283,https://www.eltiempo.com/cultura/gente/indigna...,2024-09-01T00:00:00-05:00,\n
8284,https://www.eltiempo.com/cultura/gente/murio-a...,2024-09-01T00:00:00-05:00,\n


In [5]:
# Hacemos lectura del texto XML
df_title_date_keywords_news_el_tiempo = pd.read_xml(StringIO(response.text), xpath=".//news:news", namespaces=namespaces)
df_title_date_keywords_news_el_tiempo

,publication,title,publication_date,keywords
0,\n,'Compré en Walmart de EE. UU. y me obligaron a...,2024-09-29T16:00:00-05:00,"compré, en, walmart, y, me"
1,\n,"Alias Zeus, la caída del mayor (r) que entrenó...",2024-09-29T15:58:31-05:00,"alias, zeus, la, caída, del"
2,\n,"‘Para obras de Bogotá, trabajaremos con alianz...",2024-09-29T15:57:18-05:00,"Secrtearia de Hacienda, Bogotá, endeudamiento,..."
3,\n,Duro cuestionamiento de Jaime Beltrán a Gustav...,2024-09-29T15:54:50-05:00,"duro, cuestionamiento, de, jaime, beltrán"
4,\n,El clásico Atlético vs. Real Madrid estuvo sus...,2024-09-29T15:53:07-05:00,"Real Madrid, Atlético"
...,...,...,...,...
8281,\n,"Copenhague, nuevo epicentro de la moda mundial...",2024-09-01T00:00:00-05:00,"copenhague, nuevo, epicentro, de, la"
8282,\n,Críticas al presidente Gustavo Petro por compa...,2024-09-01T00:00:00-05:00,"Gustavo Petro, IVán Duque"
8283,\n,Indignación por dos mujeres que encerraron a u...,2024-09-01T00:00:00-05:00,"Niña, avión, maltrato, mujeres, China, video, ..."
8284,\n,Murió a los 53 años el rapero Fatman Scoop tra...,2024-09-01T00:00:00-05:00,"Fatman Scoop, muerte, rapero, escenario, conci..."


In [6]:
df_urls_news_el_tiempo = (
    df_urls_news_el_tiempo[["loc"]]
    .merge(
        df_title_date_keywords_news_el_tiempo.drop(columns=["publication"]),
        left_index=True,
        right_index=True        
    )
    .rename(columns={"loc": "url_page"})
)

df_urls_news_el_tiempo

,url_page,title,publication_date,keywords
0,https://www.eltiempo.com/mundo/eeuu-y-canada/c...,'Compré en Walmart de EE. UU. y me obligaron a...,2024-09-29T16:00:00-05:00,"compré, en, walmart, y, me"
1,https://www.eltiempo.com/justicia/investigacio...,"Alias Zeus, la caída del mayor (r) que entrenó...",2024-09-29T15:58:31-05:00,"alias, zeus, la, caída, del"
2,https://www.eltiempo.com/bogota/secretaria-de-...,"‘Para obras de Bogotá, trabajaremos con alianz...",2024-09-29T15:57:18-05:00,"Secrtearia de Hacienda, Bogotá, endeudamiento,..."
3,https://www.eltiempo.com/colombia/santander/du...,Duro cuestionamiento de Jaime Beltrán a Gustav...,2024-09-29T15:54:50-05:00,"duro, cuestionamiento, de, jaime, beltrán"
4,https://www.eltiempo.com/deportes/futbol-inter...,El clásico Atlético vs. Real Madrid estuvo sus...,2024-09-29T15:53:07-05:00,"Real Madrid, Atlético"
...,...,...,...,...
8281,https://www.eltiempo.com/vida/tendencias/copen...,"Copenhague, nuevo epicentro de la moda mundial...",2024-09-01T00:00:00-05:00,"copenhague, nuevo, epicentro, de, la"
8282,https://www.eltiempo.com/politica/gobierno/cri...,Críticas al presidente Gustavo Petro por compa...,2024-09-01T00:00:00-05:00,"Gustavo Petro, IVán Duque"
8283,https://www.eltiempo.com/cultura/gente/indigna...,Indignación por dos mujeres que encerraron a u...,2024-09-01T00:00:00-05:00,"Niña, avión, maltrato, mujeres, China, video, ..."
8284,https://www.eltiempo.com/cultura/gente/murio-a...,Murió a los 53 años el rapero Fatman Scoop tra...,2024-09-01T00:00:00-05:00,"Fatman Scoop, muerte, rapero, escenario, conci..."


## ***Obtener el texto de las noticias***

In [7]:
from bs4 import BeautifulSoup

In [8]:
# Ejemplo de url a visitar:
df_urls_news_el_tiempo.iloc[0].loc["url_page"]

'https://www.eltiempo.com/mundo/eeuu-y-canada/compre-en-walmart-y-me-obligaron-a-pasar-por-estos-3-filtros-3385099'

In [9]:
%%time
# Primeras 20 urls de noticias.
list_urls = df_urls_news_el_tiempo["url_page"].head(5).to_list()

for url in list_urls:
    print("Trabajando en la URL:", url)
    request = requests.get(url)
    soup = BeautifulSoup(request.content, "html.parser")
    
    # Verificar si el request es exitosa
    if request.raise_for_status:
        author_tag = soup.find("a", class_="c-detail__author__name").get_text()
        content = " ".join([tag.get_text() for tag in soup.find_all("div", class_="paragraph")])     
        print(f"Autor: {author_tag}")
        print(f"Contenido noticia: {content}")
        print()

Trabajando en la URL: https://www.eltiempo.com/mundo/eeuu-y-canada/compre-en-walmart-y-me-obligaron-a-pasar-por-estos-3-filtros-3385099
Autor: Sol Lujan
Contenido noticia: Las cadenas de supermercado de venta minorista suelen pedirle a sus clientes el recibo de compra al salir del recinto como una medida de precaución para evitar delitos como el robo. En ese contexto, una clienta de Walmart salió ofendida del lugar cuando la obligaron a corroborar su compra no sólo una, sino tres veces.  La compradora documentó el mal momento en la tienda y lo publicó en su cuenta de TikTok @fly.girl333, donde se fue viral con más de 3 millones de reproducciones. En el vídeo se puede ver a una empleada del local vestida de civil, quien le pide el recibo por tercera vez. Aunque, la tiktoker no mencionó en dónde estaba ubicada la tienda.   La tiktoker de nombre 'Ty' publicó el vídeo sobre el mal momento que pasó en una tienda de Walmart después de comprar dos televisores y otros artículos más. Al intenta

## ***Paralelizar la tarea de obtencion del contenido de las noticias***

In [10]:
def get_content_news_from_url(url: str) -> dict:
    """
    Obtiene el contenido de una noticia desde una URL, extrayendo el autor y el cuerpo de la noticia.

    :param url: URL de la página de la noticia.
    :return: Diccionario con la URL, autor y contenido de la noticia.
    :raises ValueError: Si hay un problema con el status_code de la solicitud.
    """
    session = requests.Session()
    request = session.get(url)
    
    if request.raise_for_status:
        soup = BeautifulSoup(request.content, "html.parser")
        author_tag = soup.find("a", class_="c-detail__author__name").get_text()
        content = " ".join([tag.get_text() for tag in soup.find_all("div", class_="paragraph")])
        
        return {
            "url_page": url,
            "autor": author_tag,
            "news_content": content            
        }
    else:
        ValueError(
            f"Problemas con el status_code: {request.status_code}"
        )
        

get_content_news_from_url(df_urls_news_el_tiempo.iloc[0].loc["url_page"]) 

{'url_page': 'https://www.eltiempo.com/mundo/eeuu-y-canada/compre-en-walmart-y-me-obligaron-a-pasar-por-estos-3-filtros-3385099',
 'autor': 'Sol Lujan',
 'news_content': 'Las cadenas de supermercado de venta minorista suelen pedirle a sus clientes el recibo de compra al salir del recinto como una medida de precaución para evitar delitos como el robo. En ese contexto, una clienta de Walmart salió ofendida del lugar cuando la obligaron a corroborar su compra no sólo una, sino tres veces.\xa0 La compradora documentó el mal momento en la tienda y lo publicó en su cuenta de TikTok @fly.girl333, donde se fue viral con más de 3 millones de reproducciones. En el vídeo se puede ver a una empleada del local vestida de civil, quien le pide el recibo por tercera vez. Aunque, la tiktoker no mencionó en dónde estaba ubicada la tienda.\xa0\xa0 La tiktoker de nombre \'Ty\' publicó el vídeo sobre el mal momento que pasó en una tienda de Walmart después de comprar dos televisores y otros artículos más. 

In [11]:
import os
from typing import Dict
from concurrent.futures import ThreadPoolExecutor, as_completed

num_cores = os.cpu_count()
print(f"Número de cores disponibles: {num_cores}")

# Numero de cores a usar.
num_cores = num_cores - 2

def process_url(url: str) -> Dict:
    return get_content_news_from_url(url)


def parallel_process_urls(urls: list[str]) -> list[Dict]:
    """
    Procesa múltiples URLs en paralelo utilizando ThreadPoolExecutor,
    dejando 2 cores libres.
    
    :param urls: Lista de URLs a procesar.
    :return: Lista de diccionarios con la información extraída de cada URL.
    """
    results = []
    
    with ThreadPoolExecutor(max_workers=num_cores) as executor:
        futures = [executor.submit(process_url, url) for url in urls]
        
        for future in as_completed(futures):
            try:
                results.append(future.result())
            except Exception as e:
                print(f"Error procesando la URL: {e}")
    
    return results

# Lista de URLs a procesar. Descomente la siguiente linea de codigo 
# para obtener todas las urls.
# Procesamos todas las URLs en paralelo
# urls_to_process = df_urls_news_el_tiempo["url_page"].tolist()

# urls de ejemplo, solo una muestra de 20 urls
urls_to_process = df_urls_news_el_tiempo["url_page"].sample(20).tolist()
# Procesamos todas las URLs en paralelo
processed_results = parallel_process_urls(urls_to_process)
# Resultados
print(processed_results)
pd.DataFrame(processed_results)

Número de cores disponibles: 16
[{'url_page': 'https://www.eltiempo.com/colombia/barranquilla/capturan-a-hombre-por-presuntamente-cometer-acto-sexual-violento-con-menor-de-14-anos-en-atlantico-3379462', 'autor': 'Deivis Jhoan López Ortega', 'news_content': 'El departamento de Policía del Atlántico reportó este lunes 9 de septiembre la captura de un hombre, de 45 años de edad, por el delito de acto sexual violento con menor de 14 años, agravado. (Lea también: Video | Gobernador de Córdoba les llevó Sombrero Vueltiao a jugadores de Colombia en Barranquilla: ‘Pa’ que no usen más el chino’) De acuerdo con la información de las autoridades, se trata de una demanda de explotación sexual comercial de persona menor de 18 años de edad, consignado en el artículo 209 del Código de Policía. La aprehensión fue posible con unidades adscritas al Nuevo Modelo del Servicio de Policía orientado a las personas y los territorios y mediante planes de registro, control e identificación de personas, realizad

,url_page,autor,news_content
0,https://www.eltiempo.com/colombia/barranquilla...,Deivis Jhoan López Ortega,El departamento de Policía del Atlántico repor...
1,https://www.eltiempo.com/bogota/murio-una-pers...,Sebastián David García Castro,Una persona murió en la autopista de Sur de Bo...
2,https://www.eltiempo.com/mundo/latinoamerica/u...,Agencia EFE,Un juez de un tribunal de apelaciones revocó e...
3,https://www.eltiempo.com/politica/proceso-de-p...,Juan Sebastián Lombo Delgado,"Con dos trinos, ‘Antonio García’, jefe del Eln..."
4,https://www.eltiempo.com/mundo/latinoamerica/m...,Noticias GDA,"El angustiante llamado de José Alberto, un fot..."
5,https://www.eltiempo.com/mundo/eeuu-y-canada/p...,Pedro Ioskyn,"Este 5 de septiembre, el precio del dólar en M..."
6,https://www.eltiempo.com/mundo/eeuu-y-canada/m...,Ariadna Cruz,Actualmente en México se está discutiendo una ...
7,https://www.eltiempo.com/deportes/futbol-inter...,Redacción deportes,Hace algunos días se hizo viral el estado físi...
8,https://www.eltiempo.com/mundo/eeuu-y-canada/p...,Pedro Ioskyn,"Este 17 de septiembre, el precio del dólar en ..."
9,https://www.eltiempo.com/mundo/europa/adolesce...,Ivy Larrarte Alarcón,Un incidente violento sacudió hoy a una escuel...


In [12]:
df_news_el_tiempo = df_urls_news_el_tiempo.merge(
    pd.DataFrame(processed_results),
    on=["url_page"]    
)

display(df_news_el_tiempo)
# Descomente las siguientes lineas de codigo se desea guarda los resultados
# en un archivo parquet
# df_news_el_tiempo.to_parquet("data/df_noticias_el_tiempo.parquet")
# pd.read_parquet("data/df_noticias_el_tiempo.parquet")

,url_page,title,publication_date,keywords,autor,news_content
0,https://www.eltiempo.com/bogota/bogota-se-suma...,Bogotá se suma a la celebración del Día Mundia...,2024-09-29T12:30:55-05:00,"bogotá, se, suma, a, la",redacción bogotá,Este viernes 27 de septiembre se celebra el Dí...
1,https://www.eltiempo.com/justicia/cortes/la-es...,La estrategia que usó la defensa de un ‘narco ...,2024-09-27T06:28:05-05:00,"narcotráfico, extradición",Jesús Antonio Blanquicet,"El pasado 6 de agosto, la Corte Suprema de Jus..."
2,https://www.eltiempo.com/deportes/futbol-inter...,"'Earthing', la técnica que practica el DT de P...",2024-09-26T12:00:37-05:00,"Earthing, psg, luis enrique, técnico",Redacción deportes,Hace algunos días se hizo viral el estado físi...
3,https://www.eltiempo.com/bogota/familia-busca-...,Familia busca repatriar cuerpo de Johan Nicolá...,2024-09-24T18:20:32-05:00,"familia, repatriar, joven, estudiante, argenti...",Elim Johana Alonso Dorado,"Este 20 de septiembre, una noticia inesperada ..."
4,https://www.eltiempo.com/mundo/latinoamerica/u...,Un juez revoca la orden de prisión contra el p...,2024-09-24T16:12:10-05:00,"un, juez, revoca, la, orden",Agencia EFE,Un juez de un tribunal de apelaciones revocó e...
5,https://www.eltiempo.com/bogota/murio-una-pers...,Murió una persona atropellada por un bus de Tr...,2024-09-24T11:00:00-05:00,"Accidente, TransMilenio, portal del Sur, accid...",Sebastián David García Castro,Una persona murió en la autopista de Sur de Bo...
6,https://www.eltiempo.com/mundo/latinoamerica/m...,Madre de una joven con una glándula tumoral al...,2024-09-23T10:11:52-05:00,"Madre, Joven, Glándula tumoral, Corazón, Pulmó...",Noticias GDA,"El angustiante llamado de José Alberto, un fot..."
7,https://www.eltiempo.com/justicia/delitos/por-...,Por el magnicidio de Carlos Pizarro llaman a j...,2024-09-21T10:22:07-05:00,"Carlos Pizarro, Carlos Pizarro Leongómez, Manu...",María Isabel Ortiz Fonnegra,Más de 30 años después del asesinato del excan...
8,https://www.eltiempo.com/politica/proceso-de-p...,La polémica respuesta de 'Antonio García' al p...,2024-09-19T11:12:37-05:00,"Eln, Gobierno Petro, paz total",Juan Sebastián Lombo Delgado,"Con dos trinos, ‘Antonio García’, jefe del Eln..."
9,https://www.eltiempo.com/mundo/medio-oriente/l...,Lo que se sabe sobre las compañías de Japón y ...,2024-09-19T09:04:42-05:00,"Líbano, Hezbolá, bíperes, walkie-talkies, expl...",efe,La compañía japonesa Icom y la taiwanesa Gold ...


## ***Usar un LLM local***

In [13]:
# Instalar Ollama (disponible para Windows, Linux y MacOS)
# Aca se debe poner la guia de instalacion en Colab (pendiente)

#### ***Descargar los modelos a usar:***

1. llama3.2:3b-instruct-q8_0 
2. llama3.1:8b
3. qwen2.5:7b-instruct-q8_0

In [14]:
# Descargar los modelos:
# 1. llama3.2:3b-instruct-q8_0 --> (ollama pull llama3.2:3b-instruct-q8_0)
# 2. llama3.1:8b --> (ollama pull llama3.1:8b)
# 3. qwen2.5:7b-instruct-q8_0 --> (ollama pull qwen2.5:7b-instruct-q8_0)

In [15]:
# Descomente estas lineas para desacrgar los modelos. Esto se debe hacer
# luego de servir a Ollama (ollama serve) en terminal
# !ollama pull llama3.1:8b
# !ollama pull llama3.2:3b-instruct-q8_0

In [16]:
# Generar un chat con un llm_ollama
from langchain_ollama import ChatOllama

llm_ollama = ChatOllama(
    model="llama3.2:3b-instruct-q8_0",
    temperature=0.1,
    num_predict=1024,
)

llm_response = llm_ollama.invoke("¿Cuál es la segunda letra del alfabeto griego?")
print(llm_response)

content='La segunda letra del alfabeto griego es Beta (Β).' additional_kwargs={} response_metadata={'model': 'llama3.2:3b-instruct-q8_0', 'created_at': '2024-09-30T02:14:13.456765555Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 7344889364, 'load_duration': 7017583590, 'prompt_eval_count': 39, 'prompt_eval_duration': 39547000, 'eval_count': 15, 'eval_duration': 244202000} id='run-a42d5b5b-b69e-4606-bb03-2ff640778d24-0' usage_metadata={'input_tokens': 39, 'output_tokens': 15, 'total_tokens': 54}


In [17]:
%%time
llm_ollama.invoke(
    """Dame un lista de 20 personajes influyentes en la actualidad"""
    """mundial. No olvides incluir diferentes angulos de la vida. """
    """Solo debes darme la lista sin decir nada mas. Por ejemplo si """
    """decides incluir a Pepito Perez (suponiendo que es influyente """
    """actualmente) respuesta debe ser: Pepito Perez"""
    )

CPU times: user 98.2 ms, sys: 12.3 ms, total: 111 ms
Wall time: 2.2 s


AIMessage(content='1. Elon Musk\n2. Jeff Bezos\n3. Bill Gates\n4. Mark Zuckerberg\n5. Warren Buffett\n6. Oprah Winfrey\n7. Richard Branson\n8. Ariana Grande\n9. Cristiano Ronaldo\n10. Lionel Messi\n11. Greta Thunberg\n12. Alexandria Ocasio-Cortez\n13. Elon Musk\n14. Taylor Swift\n15. Kanye West\n16. Beyoncé\n17. LeBron James\n18. Serena Williams\n19. Malala Yousafzai\n20. Jacinda Ardern', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b-instruct-q8_0', 'created_at': '2024-09-30T02:14:15.673252447Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2195844083, 'load_duration': 43466230, 'prompt_eval_count': 97, 'prompt_eval_duration': 32951000, 'eval_count': 119, 'eval_duration': 2076684000}, id='run-0f482883-7ced-4c09-9ffa-89ba54cd1a0a-0', usage_metadata={'input_tokens': 97, 'output_tokens': 119, 'total_tokens': 216})

In [18]:
# PromptTemplate y respuesta como string
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

prompt = ChatPromptTemplate(
    [
        SystemMessagePromptTemplate.from_template(
            """Eres una util AI bot que crea excelentes biografias de """
            """personajes influyentes de la actualidad mundial y detalla su """
            """vida en diferentes aspectos. Responde usando solo JSON con """
            """llave el nombre del personaje y valor su biografia, """
            """Por ejemplo: dict(Pepito Perez="Pepito Peres ... (biografia))"""),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

print(prompt)

chain = prompt | llm_ollama | StrOutputParser()
chain.invoke(
    {
        "user_input": "Barack Obama"
    }
)

input_variables=['user_input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres una util AI bot que crea excelentes biografias de personajes influyentes de la actualidad mundial y detalla su vida en diferentes aspectos. Responde usando solo JSON con llave el nombre del personaje y valor su biografia, Por ejemplo: dict(Pepito Perez="Pepito Peres ... (biografia))'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})]


'```json\n{\n  "nombre": "Barack Obama",\n  "nacimiento": "4 de agosto de 1961",\n  "partido_politico": "Demócrata",\n  "cargos_elegidos": [\n    "Senador por Illinois (2005-2008)",\n    "Presidente de los Estados Unidos (2009-2017)"\n  ],\n  "cargos_nominados": [\n    "Presidente de los Estados Unidos (2008, 2012)"\n  ],\n  "educación": [\n    "Colegio community de Chicago (1979-1981)",\n    "Universidad de Columbia (1981-1983)",\n    "Harvard Law School (1988-1991)"\n  ],\n  "trabajo": [\n    "Abogado",\n    "Senador por Illinois",\n    "Presidente de los Estados Unidos"\n  ],\n  "logros": [\n    "Primero presidente afroamericano de los Estados Unidos",\n    "Líder en la lucha contra el cambio climático y la salud",\n    "Reforma de la ley de seguros de salud (Obamacare)"\n  ],\n  "premios": [\n    "Premio Nobel de la Paz (2009)",\n    "Medalla Presidencial de la Libertad (2017)"\n  ]\n}\n```'

In [19]:
# Usamos JSON-format para parsear la respuesta del LLM
from langchain_core.output_parsers import JsonOutputParser

json_llm = ChatOllama(
    model="llama3.2:3b-instruct-q8_0",
    temperature=0.1,
    num_predict=-1,
    format="json"
)

prompt = ChatPromptTemplate(
    [
        SystemMessagePromptTemplate.from_template(
            """Eres una util AI bot que crea excelentes biografias de """
            """personajes influyentes de la actualidad mundial y detalla su """
            """vida ampliamente en diferentes aspectos. No debes escatimar """
            """en la longitud del texto de resumen. Responde usando solo JSON """
            """format con llave el nombre del personaje y valor su biografia"""
            """Por ejemplo: dict(Pepito Perez="Pepito Perez ... (biografia)","""
            """ Benito Camelas="Benito Camelas ... (biografia)"). Debes """
            """seguir al pie de la letra el formato"""
            ),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

chain = prompt | json_llm | JsonOutputParser()
llm_answer = chain.invoke({"user_input": "Dame la biografia de 2 personajes"})
print(llm_answer, type(llm_answer))

{'Elon Musk': 'Elon Musk es un empresario y inventor sudafricano-estadounidense nacido el 28 de junio de 1971 en Pretoria, Sudáfrica. Es conocido por ser el CEO de varias empresas innovadoras, como PayPal, SpaceX, Tesla, Neuralink y The Boring Company. Musk ha sido un pionero en la industria tecnológica, impulsando avances significativos en áreas como la electricidad, la energía renovable y la exploración espacial. Su visión y liderazgo han inspirado a generaciones de innovadores y empresarios. Además, es conocido por su presencia en las redes sociales y sus declaraciones públicas sobre temas como el futuro de la humanidad y la importancia de la investigación científica.', 'Malala Yousafzai': 'Malala Yousafzai es una activista pakistaní nacida el 12 de julio de 1997 en Mingora, Pakistán. Es conocida por ser la primera niña del mundo a sobrevivir a un intento de asesinato por parte del Talibán debido a su lucha por los derechos de las mujeres en Pakistán. Malala ha sido una voz poderosa

### ***Embeddings***

In [20]:
%%time
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="llama3.1:8b",
)

question = ["¿Cuál es la segunda letra del alfabeto griego?"]
embed_question = ollama_embeddings.embed_documents(question)
print("Embeddings:")
pd.DataFrame(embed_question, index=question)

Embeddings:
CPU times: user 342 ms, sys: 0 ns, total: 342 ms
Wall time: 10.4 s


,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
¿Cuál es la segunda letra del alfabeto griego?,-0.038617,-0.01793,0.017323,0.004181,0.009179,-0.01658,-0.014371,-0.017474,0.009475,0.004823,...,-0.016437,-0.001968,-0.012676,0.011632,0.002161,-0.016913,0.020603,0.005834,0.013608,-0.006529


In [21]:
# Ejemplo: (Relevancia - Distancia)
from sklearn.metrics.pairwise import cosine_similarity
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="llama3.1:8b",
)

texts = [
    "Alpha es la primera letra del alfabeto griego",
    "A es la primera letra del alfabeto latino",
    "Beta es la segunda letra del alfabeto griego",
    "B es la segunda letra del alfabeto latino"
]
query = ["¿Cuál es la segunda letra del alfabeto griego?"]
embeds = ollama_embeddings.embed_documents(texts)
embed_query = ollama_embeddings.embed_documents(query)
print(f"Long Ollama Embeddings: {len(embed_query[0])}")
print(f"Cosine Similarities: \n {cosine_similarity(embeds, embed_query)}")

pd.DataFrame(
    cosine_similarity(embeds, embed_query),
    index=texts,
    columns=query
).sort_values(by=query, ascending=False)

Long Ollama Embeddings: 4096
Cosine Similarities: 
 [[0.59002044]
 [0.40212877]
 [0.59697701]
 [0.36079543]]


,¿Cuál es la segunda letra del alfabeto griego?
Beta es la segunda letra del alfabeto griego,0.596977
Alpha es la primera letra del alfabeto griego,0.590020
A es la primera letra del alfabeto latino,0.402129
B es la segunda letra del alfabeto latino,0.360795


In [22]:
# Embedding de las noticias.
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="llama3.1:8b",
)

df_temp = df_news_el_tiempo.sample(20)
embed_news = ollama_embeddings.embed_documents(df_temp["news_content"].to_list())
print("Embeddings:")
pd.DataFrame(embed_news, index=df_temp["url_page"])

Embeddings:


,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
url_page,,,,,,,,,,,,,,,,,,,,,
https://www.eltiempo.com/bogota/familia-busca-repatriar-a-johan-nicolas-salas-estudiante-colombiano-quien-aparecio-muerto-en-argentina-3384196,-0.008990,-0.020047,0.022114,0.023075,0.036286,-0.020619,0.000266,-0.006657,-0.002886,-0.000495,...,0.004503,-0.009030,-0.004127,0.010695,-0.014102,0.015510,0.022537,0.009786,0.001488,-0.005560
https://www.eltiempo.com/colombia/otras-ciudades/en-vivo-paro-de-camioneros-en-colombia-hoy-3-de-septiembre-bloqueos-y-como-avanza-3377480,-0.015578,0.008018,-0.010609,0.027294,0.019328,-0.035555,-0.010587,0.009581,0.022400,0.004416,...,-0.004417,0.007847,0.022179,0.005290,-0.013543,0.018618,-0.007904,0.001845,0.013003,-0.006491
https://www.eltiempo.com/mundo/medio-oriente/lo-que-dicen-las-companias-a-las-que-se-les-atribuyen-los-walkie-talkies-y-biperes-de-integrantes-de-hezbola-que-explotaron-en-el-libano-3382547,-0.018430,-0.025547,0.013802,0.012990,0.012267,-0.039588,-0.004277,-0.005028,-0.013623,0.022206,...,-0.009657,-0.012789,0.015931,-0.001338,-0.023360,0.004214,0.024876,-0.006796,0.001958,-0.007763
https://www.eltiempo.com/mundo/eeuu-y-canada/precio-del-dolar-en-mexico-a-cuanto-cotiza-hoy-5-de-septiembre-3378248,-0.024513,-0.023439,0.015425,0.021229,0.003157,-0.043095,-0.010173,-0.013166,-0.007351,0.007912,...,-0.014531,-0.014373,0.013151,0.013063,-0.012039,0.017498,0.030224,0.013202,0.009110,-0.017724
https://www.eltiempo.com/justicia/cortes/la-estrategia-que-uso-la-defensa-de-un-narco-puro-para-evitar-su-extradicion-a-ee-uu-el-hombre-cayo-veraneando-en-cartagena-con-joyas-avaluadas-en-mil-millones-de-pesos-3384974,0.000391,-0.010994,-0.018796,0.015429,-0.004247,-0.019371,-0.000085,-0.000306,-0.001966,-0.000916,...,-0.001081,0.000894,0.000448,0.015632,-0.020515,0.033598,0.013889,0.016705,0.021585,-0.015520
https://www.eltiempo.com/bogota/piscinas-gratuitas-en-bogota-son-siete-espacios-habilitados-donde-quedan-3381759,-0.017862,-0.030176,0.014179,0.035251,0.013973,-0.038072,-0.019280,-0.005178,0.008208,0.004633,...,-0.014100,-0.012323,0.002424,0.014567,0.002973,0.016548,0.029494,0.008951,0.000857,-0.022389
https://www.eltiempo.com/bogota/murio-una-persona-por-accidente-con-bus-de-transmilenio-en-autopista-sur-de-bogota-estaciones-de-bosa-soacha-y-mas-estan-sin-servicio-3383964,-0.003250,-0.012843,0.021200,0.033227,0.029351,-0.021709,-0.006312,-0.004936,0.007411,-0.003857,...,-0.002706,-0.012675,0.002485,0.010333,-0.006217,0.013824,0.015917,0.020227,0.005590,-0.014890
https://www.eltiempo.com/bogota/racionamiento-de-agua-en-bogota-estos-son-los-barrios-que-tendran-cortes-este-martes-3-de-septiembre-3374102,-0.003036,-0.016046,-0.014153,0.027828,-0.011032,-0.014027,-0.037648,0.006576,0.005396,-0.007593,...,0.015736,-0.002278,0.004887,-0.005413,-0.000571,0.015438,-0.014360,0.028526,0.013086,0.010953
https://www.eltiempo.com/cultura/gente/duenos-de-perritos-mejores-amigos-les-hicieron-videollamada-y-su-reaccion-enternecio-en-redes-sociales-3377785,-0.010963,-0.015769,0.019087,0.022672,0.039079,-0.035623,-0.005487,-0.011864,-0.006496,0.002276,...,0.000291,-0.014727,-0.004445,0.016650,-0.006778,0.009465,0.014124,0.023780,-0.001133,-0.011224


## ***K-means sobre los embeddings***

- Esta parte esta en construccion.

In [23]:
# k-means sobre los embeddings
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [24]:
scaled_data = StandardScaler().fit_transform(embed_news)
kmeans = KMeans(n_clusters=4, random_state=0, n_init="auto").fit(scaled_data)
df_temp["cluster"] = kmeans.labels_
display(df_temp["cluster"].value_counts())
df_temp

cluster
3    8
1    8
0    3
2    1
Name: count, dtype: int64

,url_page,title,publication_date,keywords,autor,news_content,cluster
3,https://www.eltiempo.com/bogota/familia-busca-...,Familia busca repatriar cuerpo de Johan Nicolá...,2024-09-24T18:20:32-05:00,"familia, repatriar, joven, estudiante, argenti...",Elim Johana Alonso Dorado,"Este 20 de septiembre, una noticia inesperada ...",3
18,https://www.eltiempo.com/colombia/otras-ciudad...,"EN VIVO, paro de camioneros en Colombia: conti...",2024-09-03T04:37:53-05:00,"en, vivo, paro, de, camioneros",REDACCIÓN ÚLTIMAS NOTICIAS,Después de varios días de intensas protestas l...,3
9,https://www.eltiempo.com/mundo/medio-oriente/l...,Lo que se sabe sobre las compañías de Japón y ...,2024-09-19T09:04:42-05:00,"Líbano, Hezbolá, bíperes, walkie-talkies, expl...",efe,La compañía japonesa Icom y la taiwanesa Gold ...,1
16,https://www.eltiempo.com/mundo/eeuu-y-canada/p...,Precio del dólar en México: ¿a cuánto cotiza h...,2024-09-05T08:18:56-05:00,"precio, del, dólar, en, méxico",Pedro Ioskyn,"Este 5 de septiembre, el precio del dólar en M...",1
1,https://www.eltiempo.com/justicia/cortes/la-es...,La estrategia que usó la defensa de un ‘narco ...,2024-09-27T06:28:05-05:00,"narcotráfico, extradición",Jesús Antonio Blanquicet,"El pasado 6 de agosto, la Corte Suprema de Jus...",0
10,https://www.eltiempo.com/bogota/piscinas-gratu...,Piscinas gratuitas en Bogotá: son siete espaci...,2024-09-17T10:15:49-05:00,"Boogtá, Piscinas gratuitas, Recomendaciones",María Camila Salas Valencia,"En Bogotá, están habilitadas siete piscinas pú...",1
5,https://www.eltiempo.com/bogota/murio-una-pers...,Murió una persona atropellada por un bus de Tr...,2024-09-24T11:00:00-05:00,"Accidente, TransMilenio, portal del Sur, accid...",Sebastián David García Castro,Una persona murió en la autopista de Sur de Bo...,3
19,https://www.eltiempo.com/bogota/racionamiento-...,Racionamiento de agua en Bogotá: estos son los...,2024-09-03T00:00:00-05:00,"Racionamiento, agua, Bogotá, barrios, afectado...",Johan Steven Guerrero,"Desde el pasado mes de abril, Bogotá ha implem...",2
17,https://www.eltiempo.com/cultura/gente/duenos-...,Dueños de perritos 'mejores amigos' les hicier...,2024-09-04T00:00:00-05:00,"Tiktok,perros, amistad, amigos, videollamada",Wendys Loraine Pitre Ariza,Hay un adagio muy popular que dice que el perr...,3
11,https://www.eltiempo.com/colombia/otras-ciudad...,¡Cuidado al conducir! Pico y placa en Pereira ...,2024-09-17T09:06:11-05:00,"Pereira, Risaralda, Pico y Placa, Horarios, Mo...",María Camila Salas Valencia,La medida de pico y placa en Pereira continúa ...,1


In [31]:
llm_ollama = ChatOllama(
    model="llama3.2:3b-instruct-q8_0",
    temperature=0.1,
    num_predict=2048,
    num_ctx=32000
)

prompt_caracterizacion = ChatPromptTemplate(
    [
        SystemMessagePromptTemplate.from_template(
            """Eres una util AI bot que identica las tematicas generales de """
            """las noticias que te van a proporcionar pues estas noticias se """
            """supone estas relacionadas por sus embeddings. La respuesta de los """
            """temas deben ser similitudes precisamente semanticas, debes dar """
            """tu respuesta con el titulo que determina porque son similares estas """
            """noticias ademas de 5 bullets. Finalmente, daras un resumen al estilo """
            """de historia de todas las noticias para poner al dia en noticias """
            """ de la tematicas que las hace coincidentes al usuario. Un resumen"""
            """ por cada noticia. Debes decirme cuantas noticias te fueron """
            """proporcionadas"""         
            ),
        HumanMessagePromptTemplate.from_template("{news}")
    ]
)

llm_chain = prompt_caracterizacion | llm_ollama | StrOutputParser()

In [26]:
from pprint import pprint

list_news = df_temp.query("cluster == 0")["news_content"].to_list()
dict_news_content = {f"noticia_{i+1}": list_news[i] for i in range(len(list_news))}
pprint(dict_news_content)

{'noticia_1': 'El pasado 6 de agosto, la Corte Suprema de Justicia avaló la '
              'extradición de un señalado narcotraficante que fue capturado en '
              'enero de 2023 en un exclusivo sector de Cartagena, hasta donde '
              'llegó para pasar las fiestas de fin de año. (Lea:\xa0Las claves '
              'del fallo de la Corte que declaró a los animales de compañía '
              'inembargables dentro de un proceso judicial) El Gobierno de los '
              'Estados Unidos, a través de su Embajada en Colombia, mediante '
              'nota verbal del 31 de agosto de 2022, había solicitado la '
              'detención preventiva con fines de extradición del ciudadano '
              'colombiano, al ser requerido por  la Corte Distrital de Florida '
              '– División Tampa, por delitos relacionados con tráfico de '
              'drogas ilícitas y concierto para delinquir.  Se trata de Javer '
              'Alexis López Murillo, alias Toyota, qui

In [32]:
import json

result = json.dumps(dict_news_content)
response_cluster_0 = llm_chain.invoke({"news": result})
print(response_cluster_0)

**Tema general: Justicia y Derechos Humanos**

Las noticias proporcionadas son relacionadas con temas de justicia y derechos humanos en Colombia. A continuación, se presentan las similitudes precisamente semanticas entre las noticias:

* **Extradición de narcotraficantes**: La primera noticia habla sobre la extradición de un narcotraficante colombiano a los Estados Unidos, mientras que la tercera noticia trata sobre la condena de un exdirector del DAS por su presunta participación en el asesinato de un excandidato presidencial.
* **Procesos de paz y violencia**: La segunda noticia se centra en la suspensión de los diálogos de paz entre el gobierno colombiano y el grupo guerrillero Eln, mientras que la primera noticia menciona la operación militar contra el narcotraficante extraditado.
* **Protección de derechos humanos**: La tercera noticia se refiere a la condena del exdirector del DAS por su presunta participación en el asesinato de un excandidato presidencial, lo que destaca la impo